In [8]:
import torch
print(torch.cuda.get_device_properties(0).total_memory)
c = torch.cuda.memory_cached(0)
a = torch.cuda.memory_allocated(0)
f = c-a  # free inside cache
print(f)

6442450944
0


In [9]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

total    : 6442450944
free     : 6345981952
used     : 96468992


In [1]:
import argparse
import os
import time
import pdb
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import mlflow
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

from datasets import RoomsDataset_mk2
from pointnet import PointNetSeg
from utils import get_s3d_num2cat
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [28]:
class RoomsDataset_mk3:
    def __init__(self, path, npoints=4096, slicing_sizes=(1.0, 1.0, 1.0)):
        self.path = path
        self.npoints = npoints
        self.slicing_sizes = slicing_sizes
        self.generate_boxes()
        self.size = len(self.boxes)

    def __len__(self):
            return self.size

    def generate_boxes(self):
        points = np.loadtxt(self.path, skiprows=1)[:, :3]
        self.boxes = []
        mins = np.min(points, axis=0)
        maxs = np.max(points, axis=0)
        ranges = maxs-mins
        ns=(ranges/np.array(self.slicing_sizes)).astype(np.int64)
        for ix in range(ns[0]):
            for iy in range(ns[1]):
                for iz in range(ns[2]):
                    x_indices = (mins[0]+self.slicing_sizes[0]*ix<points[:, 0]) * (mins[0]+self.slicing_sizes[0]*(ix+1)>points[:, 0])
                    y_indices = (mins[1]+self.slicing_sizes[1]*iy<points[:, 1]) * (mins[1]+self.slicing_sizes[1]*(iy+1)>points[:, 1])
                    z_indices = (mins[2]+self.slicing_sizes[2]*iz<points[:, 2]) * (mins[2]+self.slicing_sizes[2]*(iz+1)>points[:, 2])
                    indices = x_indices*y_indices*z_indices
                    if np.sum(indices) == 0:
                        continue
                    self.boxes.append(points[indices, :])

                    
    def __getitem__(self, idx):
        choice = np.random.choice(self.boxes[idx].shape[0], self.npoints, replace=True)
        return scale_linear_bycolumn(self.boxes[idx][choice, :]), self.boxes[idx][choice, :]
    
    
def scale_linear_bycolumn(rawdata, high=1.0, low=0.0):
    mins = np.min(rawdata, axis=0)
    maxs = np.max(rawdata, axis=0)
    rng = maxs - mins
    return high - (high-low)*(maxs-rawdata)/(rng+np.finfo(np.float32).eps)

def xyzl_to_xyzrgb(xyzl):
    """
        input: coordinates + label
        output: coordinates + rgb
    """
    from matplotlib import cm
    classes = np.unique(xyzl[:, -1]).astype(np.int32)
    cmap = cm.get_cmap('jet')

    colored = []
    for i, class_num in enumerate(classes):
        classed_points = xyzl[xyzl[:, -1]==class_num].astype(np.float)
        colors = np.zeros((classed_points.shape[0], 3))
        colors[:] = cmap(i / len(classes))[:-1]
        colored.append(np.concatenate([classed_points[:, :3], colors], axis=1))
        
    return np.concatenate(colored, axis=0)
    

In [3]:
num2cat = get_s3d_num2cat()
num_classes = len(list(num2cat))
classifier = PointNetSeg(k=num_classes)
classifier.load_state_dict(torch.load("Roma_s3dis3_model_10.pth"))

In [4]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
classifier = classifier.to(device)

In [5]:
dataset = RoomsDataset_mk3(path='205_3.pts', npoints=4096)
dataloader = torch.utils.data.DataLoader(dataset, 
                                         batch_size=30, 
                                         shuffle=False, 
                                         num_workers=0)

classifier = classifier.eval()

In [6]:
# CPU only
processed = []
for i, data in enumerate(dataloader):
    scaled_points, initial_points = data
    points = scaled_points.to(device).float()
    points = points.transpose(2, 1)
    pred, _ = classifier(points)
    del points
    classes = pred.view(-1, num_classes).data.max(1)[1]
    classes = classes.to("cpu").unsqueeze(0).numpy().T
    processed.append(np.concatenate([initial_points.view(-1, 3), classes], axis=1))

processed = np.concatenate(processed, axis=0)
np.savetxt("labeled.pts", processed)
colored = xyzl_to_xyzrgb(processed)
np.savetxt("colored.pts", colored)

In [11]:
processed = []
for i in tqdm(range(len(dataset))):
    initial_points, notscaled__points = dataset[i]
    points = torch.from_numpy(initial_points).to(device).float()
    points = points.unsqueeze(0)
    points = points.transpose(2, 1)
    
    pred, _ = classifier(points)
    classes = pred.view(-1, num_classes).data.max(1)[1]
    classes = classes.to("cpu").unsqueeze(0).numpy().T
    processed.append(np.concatenate([notscaled__points, classes], axis=1))
    
np.savetxt("labeled.pts", np.concatenate(processed, axis=0))

100%|███████████████████████████████████████████████████████████████████████████████| 196/196 [00:01<00:00, 107.99it/s]


In [ ]:
np.savetxt("labeled.pts", np.concatenate(processed, axis=0))